In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.append('../')

In [2]:
from sbe_vallib import Validation, BinaryScorer, BinarySampler

## Dataset

In [8]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

X, y = make_classification(n_samples=1000, n_classes=2,
                               n_features=5, n_informative=3, n_redundant=0,
                               random_state=0)

X = pd.DataFrame(X)
y = pd.Series(y)

X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)


## Model

In [25]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X_train, y_train)

## Sampler Scorer

In [26]:
from sbe_vallib import BinarySampler, BinaryScorer

sampler = BinarySampler(train={'X': X_train, 'y_true': y_train, 'y_pred': model.predict_proba(X_train)},
                        oos = {'X': X_test, 'y_true': y_test, 'y_pred': model.predict_proba(X_test)}, bootstrap=True)

scorer = BinaryScorer()

In [27]:
def custom_test(model, scorer, sampler, a = 10, **kwargs):
    metrics = []
    for i in range(a):
        sampler.set_seed(i)
        train = sampler.train
        oos = sampler.oos
        model.fit(X=train["X"], y=train["y_true"])
        y_pred = model.predict_proba(oos['X'])
        metrics.append(scorer.score(oos['y_true'], y_pred))
    
    return {
        "semaphore": "grey",
        "result_dict": {'mean_f1': np.mean([i['f1_score'] for i in metrics])},
        "result_dataframes": [pd.DataFrame(metrics)],
        "result_plots": [],
    }


custom_tests = {
    "test_0_0": {"block": "model_stability", "callable": custom_test, "params": {}},
    "test_0_0_1": {
        "block": "model_stability",
        "callable": custom_test,
        "params": {"a": 100},
    },
}


validor = Validation(model, sampler, scorer, custom_tests=custom_tests,
                         pipeline='../sbe_vallib/validation/table/pipelines/Config_31.xlsx')
res = validor.validate()
res.keys()

dict_keys(['train_test_independence_test', 'test_extremal_missing_values', 'psi_factor_classes_test_oos', 'psi_factor_classes_test_oot', 'test_key_metric', 'test_confidence_inteval', 'test_presicion', 'test_recall', 'test_2_5', 'test_3_1', 'test_3_2', 'test_feature_importance', 'test_4_2', 'test_key_metric_stability', 'test_presicion_stability', 'test_recall_stability', 'test_5_4', 'test_5_6', 'test_5_7', 'test_0_0', 'test_0_0_1'])

In [32]:
res['test_0_0_1']['result_dataframes'][0]

,accuracy_score,precision_score,recall_score,f1_score,gini
0,0.726667,0.702703,0.732394,0.717241,0.606347
1,0.696667,0.710345,0.677632,0.693603,0.633890
2,0.763333,0.726619,0.753731,0.739927,0.703830
3,0.770000,0.765101,0.770270,0.767677,0.691679
4,0.693333,0.699301,0.671141,0.684932,0.558025
...,...,...,...,...,...
95,0.730000,0.775148,0.752874,0.763848,0.635742
96,0.720000,0.681159,0.701493,0.691176,0.625787
97,0.720000,0.725926,0.675862,0.700000,0.620645
98,0.753333,0.770370,0.707483,0.737589,0.662265
